# Address Matching with ML (Sanitized, Refactored)

This notebook trains an **scikit-learn** model to match **vendor premises** to **site inventory** records using:

- Address text similarity features (street/city/state)
- House number difference
- Optional fuzzy score
- **Geo-distance features (miles)** using **LAT/LON**
- A public geocoding API (**US Census Geocoder**) to generate **vendor** LAT/LON  
  *(site inventory is assumed to already include LAT/LON)*

> Notes
> - This is company-agnostic: no proprietary paths, names, or identifiers.
> - You’ll need to map your real column names in `COLUMN_MAP`.


In [ ]:
# --- Config (edit this) ---
from pathlib import Path

DATA_DIR = Path("./data")
OUTPUT_DIR = Path("./output")
CACHE_DIR = Path("./cache")

OUTPUT_DIR.mkdir(parents=True, exist_ok=True)
CACHE_DIR.mkdir(parents=True, exist_ok=True)

# Input files
SITE_FILE  = DATA_DIR / "site_inventory.csv"
VENDOR_FILE = DATA_DIR / "vendor_premises.csv"

# Optional: labeled training pairs (recommended)
# Must contain: vendor_id, site_id, label (1 match / 0 non-match)
LABELED_PAIRS_FILE = DATA_DIR / "labeled_pairs.csv"  # set to None if you don't have labels

# Column mapping (update to your real columns)
COLUMN_MAP = {
    "site": {
        "id": "site_id",
        "street": "street",
        "city": "city",
        "state": "state",
        "zip": "zip",
        "lat": "latitude",     # assumed present in site inventory
        "lon": "longitude",
    },
    "vendor": {
        "id": "vendor_id",
        "street": "street",
        "city": "city",
        "state": "state",
        "zip": "zip",
        # vendor lat/lon will be generated via geocoding
    }
}

# Candidate generation settings
TOP_K_PER_VENDOR = 20          # number of candidate sites kept per vendor after blocking + quick scoring
FUZZY_QUICK_THRESHOLD = 70     # quick filter before feature computation (keep loose; model does final decision)

# Model settings
TEST_SIZE = 0.25
RANDOM_STATE = 42


## Imports

In [ ]:
import os
import re
import time
import math
import json
import numpy as np
import pandas as pd
import requests

from sklearn.model_selection import train_test_split
from sklearn.pipeline import Pipeline
from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import classification_report, roc_auc_score

# If you have rapidfuzz installed, it's faster/better than difflib
try:
    from rapidfuzz.fuzz import token_sort_ratio
except Exception:
    token_sort_ratio = None


## Helpers (text + geo)

In [ ]:
# --- Text normalization helpers ---
def normalize_text(x: object) -> str:
    if x is None or (isinstance(x, float) and np.isnan(x)):
        return ""
    s = str(x).strip().lower()
    s = re.sub(r"[^a-z0-9\s]", " ", s)
    s = re.sub(r"\s+", " ", s).strip()
    return s

def normalize_zip(x: object) -> str:
    s = normalize_text(x)
    s = re.sub(r"[^0-9]", "", s)
    return (s[:5] if len(s) >= 5 else s).zfill(5) if s else ""

def extract_house_number(street: str) -> float:
    street = normalize_text(street)
    m = re.match(r"^(\d+)", street)
    return float(m.group(1)) if m else np.nan

def street_core(street: str) -> str:
    # Drop house number; keep core tokens
    s = normalize_text(street)
    s = re.sub(r"^\d+\s*", "", s)
    return s

def sim(a: str, b: str) -> float:
    a, b = normalize_text(a), normalize_text(b)
    if not a and not b:
        return 100.0
    if token_sort_ratio:
        return float(token_sort_ratio(a, b))
    # fallback: simple overlap proxy (0..100)
    aset, bset = set(a.split()), set(b.split())
    if not aset and not bset:
        return 100.0
    if not aset or not bset:
        return 0.0
    return 100.0 * (len(aset & bset) / max(1, len(aset | bset)))

# --- Geo helpers ---
def haversine_miles(lat1, lon1, lat2, lon2) -> float:
    try:
        lat1, lon1, lat2, lon2 = map(float, [lat1, lon1, lat2, lon2])
    except Exception:
        return np.nan

    # Earth radius in miles
    R = 3958.7613
    phi1, phi2 = math.radians(lat1), math.radians(lat2)
    dphi = math.radians(lat2 - lat1)
    dlambda = math.radians(lon2 - lon1)

    a = math.sin(dphi/2)**2 + math.cos(phi1)*math.cos(phi2)*math.sin(dlambda/2)**2
    c = 2 * math.atan2(math.sqrt(a), math.sqrt(1-a))
    return R * c


## Public geocoding for vendor premises (with cache)

In [ ]:
# --- Vendor geocoding (public API: US Census Geocoder) ---
CACHE_PATH = CACHE_DIR / "vendor_geocode_cache.parquet"

def make_cache_key(street, city, state, zip5) -> str:
    return f"{normalize_text(street)}|{normalize_text(city)}|{normalize_text(state)}|{normalize_zip(zip5)}"

def load_cache() -> pd.DataFrame:
    if CACHE_PATH.exists():
        return pd.read_parquet(CACHE_PATH)
    return pd.DataFrame(columns=["cache_key", "lat", "lon", "status", "queried_at", "source"])

def save_cache(df: pd.DataFrame) -> None:
    df.to_parquet(CACHE_PATH, index=False)

def upsert_cache(cache_df: pd.DataFrame, new_rows_df: pd.DataFrame) -> pd.DataFrame:
    if new_rows_df is None or new_rows_df.empty:
        return cache_df
    out = pd.concat([cache_df, new_rows_df], ignore_index=True)
    out = out.drop_duplicates("cache_key", keep="last")
    return out

def geocode_one_census(street, city, state, zip5):
    """Returns (lat, lon, status) where status is OK | NO_MATCH | ERROR"""
    addr = f"{street}, {city}, {state} {zip5}".strip()
    url = "https://geocoding.geo.census.gov/geocoder/locations/onelineaddress"
    params = {"address": addr, "benchmark": "Public_AR_Current", "format": "json"}

    try:
        resp = requests.get(url, params=params, timeout=20)
        resp.raise_for_status()
        data = resp.json()
        matches = data.get("result", {}).get("addressMatches", [])
        if not matches:
            return np.nan, np.nan, "NO_MATCH"
        m = matches[0]
        lon = float(m["coordinates"]["x"])
        lat = float(m["coordinates"]["y"])
        return lat, lon, "OK"
    except Exception:
        return np.nan, np.nan, "ERROR"

def geocode_vendor_premises(vendor_df: pd.DataFrame, column_map: dict, limit=None, sleep_s=0.15) -> pd.DataFrame:
    """Adds vendor_lat, vendor_lon, vendor_geocode_status using cache + Census geocoder."""
    v = vendor_df.copy()

    v_id = column_map["vendor"]["id"]
    v_street = column_map["vendor"]["street"]
    v_city = column_map["vendor"]["city"]
    v_state = column_map["vendor"]["state"]
    v_zip = column_map["vendor"]["zip"]

    v["cache_key"] = v.apply(lambda r: make_cache_key(r[v_street], r[v_city], r[v_state], r[v_zip]), axis=1)

    cache = load_cache()
    cached_keys = set(cache["cache_key"].astype(str))
    all_keys = v["cache_key"].astype(str).drop_duplicates().tolist()
    misses = [k for k in all_keys if k not in cached_keys]

    if limit is not None:
        misses = misses[:limit]

    print("Cache misses to geocode:", len(misses))

    if misses:
        uniq = v.drop_duplicates("cache_key").set_index("cache_key")
        new_rows = []
        for i, k in enumerate(misses, start=1):
            r = uniq.loc[k]
            lat, lon, status = geocode_one_census(r[v_street], r[v_city], r[v_state], r[v_zip])
            new_rows.append({
                "cache_key": k,
                "lat": lat,
                "lon": lon,
                "status": status,
                "queried_at": pd.Timestamp.utcnow(),
                "source": "census"
            })
            if i % 25 == 0:
                print(f"  geocoded {i}/{len(misses)}")
            time.sleep(sleep_s)

        cache = upsert_cache(cache, pd.DataFrame(new_rows))
        save_cache(cache)

    # Merge cache onto vendor records
    v = v.merge(
        cache[["cache_key", "lat", "lon", "status"]],
        on="cache_key",
        how="left"
    ).rename(columns={"lat":"vendor_lat", "lon":"vendor_lon", "status":"vendor_geocode_status"})

    return v


## Candidate generation and features

In [ ]:
# --- Candidate generation + feature engineering ---
def prepare_site_inventory(site_df: pd.DataFrame, column_map: dict) -> pd.DataFrame:
    s = site_df.copy()
    m = column_map["site"]
    s["site_id"] = s[m["id"]]
    s["site_street_norm"] = s[m["street"]].map(normalize_text)
    s["site_city_norm"] = s[m["city"]].map(normalize_text)
    s["site_state_norm"] = s[m["state"]].map(normalize_text)
    s["site_zip5"] = s[m["zip"]].map(normalize_zip)
    s["site_house_num"] = s[m["street"]].map(extract_house_number)
    s["site_street_core"] = s[m["street"]].map(street_core)
    s["site_lat"] = pd.to_numeric(s[m["lat"]], errors="coerce")
    s["site_lon"] = pd.to_numeric(s[m["lon"]], errors="coerce")
    return s

def prepare_vendor_premises(vendor_df: pd.DataFrame, column_map: dict) -> pd.DataFrame:
    v = vendor_df.copy()
    m = column_map["vendor"]
    v["vendor_id"] = v[m["id"]]
    v["vendor_street_norm"] = v[m["street"]].map(normalize_text)
    v["vendor_city_norm"] = v[m["city"]].map(normalize_text)
    v["vendor_state_norm"] = v[m["state"]].map(normalize_text)
    v["vendor_zip5"] = v[m["zip"]].map(normalize_zip)
    v["vendor_house_num"] = v[m["street"]].map(extract_house_number)
    v["vendor_street_core"] = v[m["street"]].map(street_core)
    return v

def build_candidates(vendor_df: pd.DataFrame, site_df: pd.DataFrame, top_k_per_vendor=20) -> pd.DataFrame:
    """Block by (state, zip5), then keep top-k by quick street similarity."""
    # Block join
    blocked = vendor_df.merge(
        site_df,
        left_on=["vendor_state_norm","vendor_zip5"],
        right_on=["site_state_norm","site_zip5"],
        how="left",
        suffixes=("", "")
    )

    # Quick score to prune pairs early
    blocked["quick_street_sim"] = blocked.apply(
        lambda r: sim(r["vendor_street_norm"], r["site_street_norm"]), axis=1
    )

    # Optional: apply a loose quick filter
    blocked = blocked[blocked["quick_street_sim"] >= FUZZY_QUICK_THRESHOLD].copy()

    # Keep top-k per vendor_id
    blocked.sort_values(["vendor_id","quick_street_sim"], ascending=[True, False], inplace=True)
    topk = blocked.groupby("vendor_id", as_index=False).head(top_k_per_vendor).copy()
    return topk

def add_features(pairs_df: pd.DataFrame) -> pd.DataFrame:
    df = pairs_df.copy()

    # Text similarity features
    df["street_sim"] = df.apply(lambda r: sim(r["vendor_street_norm"], r["site_street_norm"]), axis=1)
    df["city_sim"]   = df.apply(lambda r: sim(r["vendor_city_norm"],  r["site_city_norm"]), axis=1)

    # Flags / numeric diffs
    df["state_match"] = (df["vendor_state_norm"] == df["site_state_norm"]).astype(int)
    df["same_street_core"] = (df["vendor_street_core"] == df["site_street_core"]).astype(int)
    df["house_diff"] = (pd.to_numeric(df["vendor_house_num"], errors="coerce") - pd.to_numeric(df["site_house_num"], errors="coerce")).abs()

    # Fuzzy score feature (if rapidfuzz not available, reuse street_sim)
    if token_sort_ratio:
        df["fuzzy_score"] = df.apply(lambda r: float(token_sort_ratio(r["vendor_street_norm"], r["site_street_norm"])), axis=1)
    else:
        df["fuzzy_score"] = df["street_sim"]

    # Geo features (requires vendor_lat/lon and site_lat/lon)
    df["has_geo"] = (
        df["vendor_geocode_status"].eq("OK") &
        df["vendor_lat"].notna() & df["vendor_lon"].notna() &
        df["site_lat"].notna() & df["site_lon"].notna()
    ).astype(int)

    df["geo_miles"] = df.apply(
        lambda r: haversine_miles(r["vendor_lat"], r["vendor_lon"], r["site_lat"], r["site_lon"]) if r["has_geo"] == 1 else np.nan,
        axis=1
    )

    return df


## Model

In [ ]:
# --- Model training / evaluation ---
FEATURE_COLS = [
    "street_sim", "city_sim",
    "state_match", "same_street_core",
    "house_diff", "fuzzy_score",
    "geo_miles", "has_geo"
]

def make_model() -> Pipeline:
    return Pipeline([
        ("imputer", SimpleImputer(strategy="median")),
        ("scaler", StandardScaler()),
        ("clf", LogisticRegression(max_iter=6000))
    ])

def train_model(training_df: pd.DataFrame) -> Pipeline:
    X = training_df[FEATURE_COLS]
    y = training_df["label"].astype(int)

    X_train, X_test, y_train, y_test = train_test_split(
        X, y, test_size=TEST_SIZE, random_state=RANDOM_STATE, stratify=y
    )

    model = make_model()
    model.fit(X_train, y_train)

    y_pred = model.predict(X_test)
    y_prob = model.predict_proba(X_test)[:, 1]

    print(classification_report(y_test, y_pred))
    print("ROC AUC:", roc_auc_score(y_test, y_prob))

    return model


## Run end-to-end

In [ ]:
# --- Load data ---
site_raw = pd.read_csv(SITE_FILE)
vendor_raw = pd.read_csv(VENDOR_FILE)

site_df = prepare_site_inventory(site_raw, COLUMN_MAP)
vendor_df = prepare_vendor_premises(vendor_raw, COLUMN_MAP)

# --- Geocode vendor premises (adds vendor_lat/vendor_lon) ---
vendor_geo_df = geocode_vendor_premises(vendor_df, COLUMN_MAP)

# --- Candidate pairs (blocked + top-k pruning) ---
candidate_pairs = build_candidates(vendor_geo_df, site_df, top_k_per_vendor=TOP_K_PER_VENDOR)

# --- Feature engineering (includes geo_miles + has_geo) ---
feature_pairs = add_features(candidate_pairs)

feature_pairs.head()


## Bring in labels and train

In [ ]:
# --- Training labels (recommended) ---
# Expect a file with: vendor_id, site_id, label (1=match, 0=no-match)
if LABELED_PAIRS_FILE is None:
    raise ValueError("Set LABELED_PAIRS_FILE to a labeled pairs CSV to train the model.")

labeled_pairs = pd.read_csv(LABELED_PAIRS_FILE)

training_df = feature_pairs.merge(
    labeled_pairs[["vendor_id","site_id","label"]],
    on=["vendor_id","site_id"],
    how="inner"
)

print("Training rows:", len(training_df))
training_df[["vendor_id","site_id","label"]].head()


In [ ]:
# --- Train + evaluate ---
model = train_model(training_df)


## Score and export

In [ ]:
# --- Score all candidate pairs + pick best per vendor ---
feature_pairs = feature_pairs.copy()
X_all = feature_pairs[FEATURE_COLS]
feature_pairs["match_probability"] = model.predict_proba(X_all)[:, 1]

best_matches = (
    feature_pairs.sort_values(["vendor_id","match_probability"], ascending=[True, False])
    .groupby("vendor_id", as_index=False)
    .head(1)
    .copy()
)

best_matches[["vendor_id","site_id","match_probability","geo_miles","street_sim","city_sim"]].head()


In [ ]:
# --- Export ---
best_out = OUTPUT_DIR / "best_vendor_to_site_matches.csv"
all_out  = OUTPUT_DIR / "scored_candidate_pairs.csv"

best_matches.to_csv(best_out, index=False)
feature_pairs.to_csv(all_out, index=False)

print("Wrote:", best_out)
print("Wrote:", all_out)
